In [1]:
#Pobieranie potrzebnych bibliotek
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Ładowanie danych i sprawdzenie kolumn
data = pd.read_excel('./artifacts/Mincer.xlsx')
print(data.columns)

#Stworzenie kolumny 'lata_edukacji'
edu_mapping = {
    0 :  8,
    1:  12,
    2:  15,
    3:  15.5,
    4:  17,
    5:  17,
    6:  19,
    7:  19,
    8:  21
}

data['edu_lata'] = data['edukacja_stp'].map(edu_mapping)
data['dosw_sqrt'] = data['doswiadczenie']**2

data = data.replace({True: 1, False: 0})
print(data.head(1))

Index(['Unnamed: 0', 'doswiadczenie', 'pensja', 'najwyzsze', 'matura',
       'doswiadczenie_log', 'czy_ma_mature', 'czy_ma_lic', 'czy_ma_inz',
       'czy_ma_mgr', 'czy_ma_mgrinz', 'czy_ma_podyplomowe', 'czy_ma_mba',
       'czy_ma_phd', 'kujawsko_pomorskie', 'lubelskie', 'lubuskie', 'lodzkie',
       'malopolskie', 'mazowieckie', 'opolskie', 'podkarpackie', 'podlaskie',
       'pomorskie', 'slaskie', 'swietokrzyskie', 'warminsko_mazurskie',
       'wielkopolskie', 'zachodniopomorskie', 'male_miasto', 'srednie_miasto',
       'duze_miasto', 'kobieta', 'ln_wyn', 'edukacja_stp'],
      dtype='object')
   Unnamed: 0  doswiadczenie  pensja  najwyzsze  matura  doswiadczenie_log  \
0           0             39    2500        NaN       0           3.688879   

   czy_ma_mature  czy_ma_lic  czy_ma_inz  czy_ma_mgr  ...  zachodniopomorskie  \
0              0           0           0           0  ...                   0   

   male_miasto  srednie_miasto  duze_miasto  kobieta    ln_wyn  edukacja

In [3]:
# Deklarowanie zmiennych potrzebnych do stworzenia modelu
zmienne_niezalezne = [
    'edu_lata',
    'doswiadczenie',
    'dosw_sqrt',
    'kobieta'
]

y = np.log(data['pensja'])

kwantyle = [0.25, 0.50, 0.75]

wyniki = []

In [4]:
# Model
for q in kwantyle:
    formula = f'y ~ edu_lata + doswiadczenie + dosw_sqrt + kobieta'
    model = smf.quantreg(formula, data=data)
    quantreg_results = model.fit(q=q)
    wyniki.append(quantreg_results)

print(wyniki)

[<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002A5C25453D0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002A5C24E9350>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002A5C2544290>]


In [5]:
# Zapisanie wyników
for i, result in enumerate(wyniki):
    with open(f'./wyniki_model4_minc_kwant/kwantyl_{kwantyle[i]}_wyniki.txt', 'w') as file:
        file.write(result.summary().as_text())

In [10]:
#VIF
new_df = data[zmienne_niezalezne].copy()

vif = pd.DataFrame()
vif["Variable"] = new_df.columns
vif["VIF"] = [variance_inflation_factor(new_df.values, i) for i in range(new_df.shape[1])]
print(vif)

        Variable       VIF
0       edu_lata  3.561506
1  doswiadczenie  2.474861
2        kobieta  2.094765


In [14]:
# Wyświetlenie estymatorów do interpretacji
decomposition_results = {}

for i, result in enumerate(wyniki):
    quantile = kwantyle[i]  
    decomposition_results[quantile] = {}
    
    for var in zmienne_niezalezne:
        effect = result.params[var]
        decomposition_results[quantile][var] = effect
for quantile, effects in decomposition_results.items():
    print(f"Quantile {quantile}:")
    for var, effect in effects.items():
        print(f"{var}: {effect:.4f}")

Quantile 0.25:
edu_lata: 0.0362
doswiadczenie: 0.0168
dosw_sqrt: -0.0004
kobieta: -0.3051
Quantile 0.5:
edu_lata: 0.0442
doswiadczenie: 0.0132
dosw_sqrt: -0.0002
kobieta: -0.3365
Quantile 0.75:
edu_lata: 0.0485
doswiadczenie: 0.0148
dosw_sqrt: -0.0003
kobieta: -0.4195
